In [2]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'

In [3]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

Opening the tests dataset

In [54]:
#opening test
book_test=spark.read.parquet('file:///home/talentum/shared/BIGDATAPROYECT/book_test.parquet/stock_id=0/*')
book_test.show()

#This is all the data for test, it is clean

+-------+-----------------+----------+----------+----------+----------+---------+---------+---------+---------+
|time_id|seconds_in_bucket|bid_price1|ask_price1|bid_price2|ask_price2|bid_size1|ask_size1|bid_size2|ask_size2|
+-------+-----------------+----------+----------+----------+----------+---------+---------+---------+---------+
|      4|                0| 1.0000491|   1.00059|0.99965584| 1.0006392|       91|      100|      100|       24|
|      4|                1| 1.0000491|   1.00059|0.99965584| 1.0006392|       91|      100|      100|       20|
|      4|                5| 1.0000491| 1.0006392|0.99965584|  1.000885|      290|       20|      101|       15|
+-------+-----------------+----------+----------+----------+----------+---------+---------+---------+---------+



Example of data 0 in the train data

In [4]:
#opening train only dataset 0
book_train=spark.read.parquet('file:///home/talentum/shared/BIGDATAPROYECT/book_train.parquet/stock_id=0/c439ef22282f412ba39e9137a3fdabac.parquet')
book_train.show(20)
book_train.columns
#31a1c5cd6d8546b383d10373db762236.parquet


+-------+-----------------+----------+----------+----------+----------+---------+---------+---------+---------+
|time_id|seconds_in_bucket|bid_price1|ask_price1|bid_price2|ask_price2|bid_size1|ask_size1|bid_size2|ask_size2|
+-------+-----------------+----------+----------+----------+----------+---------+---------+---------+---------+
|      5|                0| 1.0014222| 1.0023013| 1.0013704| 1.0023531|        3|      226|        2|      100|
|      5|                1| 1.0014222| 1.0023013| 1.0013704| 1.0023531|        3|      100|        2|      100|
|      5|                5| 1.0014222| 1.0023013| 1.0013704| 1.0024048|        3|      100|        2|      100|
|      5|                6| 1.0014222| 1.0023013| 1.0013704| 1.0024048|        3|      126|        2|      100|
|      5|                7| 1.0014222| 1.0023013| 1.0013704| 1.0024048|        3|      126|        2|      100|
|      5|               11| 1.0014222| 1.0023013| 1.0013704| 1.0024048|        3|      100|        2|   

['time_id',
 'seconds_in_bucket',
 'bid_price1',
 'ask_price1',
 'bid_price2',
 'ask_price2',
 'bid_size1',
 'ask_size1',
 'bid_size2',
 'ask_size2']

Opening all train data and analizing if is it clean and if present null values.

In [32]:
#opening all data
mergedDF = spark.read.option("mergeSchema", "true").parquet('file:///home/talentum/shared/BIGDATAPROYECT/book_train.parquet/')


In [33]:
mergedDF.show(10)
mergedDF.printSchema()



+-------+-----------------+----------+----------+----------+----------+---------+---------+---------+---------+--------+
|time_id|seconds_in_bucket|bid_price1|ask_price1|bid_price2|ask_price2|bid_size1|ask_size1|bid_size2|ask_size2|stock_id|
+-------+-----------------+----------+----------+----------+----------+---------+---------+---------+---------+--------+
|      5|                0| 1.0018843|  1.001916| 1.0018528| 1.0019478|      200|      100|      400|     1089|      29|
|      5|                1| 1.0021378| 1.0021694| 1.0021061| 1.0022011|      100|      900|      300|     1089|      29|
|      5|                2| 1.0021378| 1.0022011| 1.0021061| 1.0022328|      328|      289|      228|      989|      29|
|      5|                3| 1.0021061| 1.0021378| 1.0020744| 1.0021694|      328|      100|     1028|     1089|      29|
|      5|                4| 1.0020111| 1.0020744| 1.0019794| 1.0021061|      228|     1089|     1200|     1589|      29|
|      5|                5| 1.00

In [37]:
idi=mergedDF.select('stock_id').distinct().count() 
print(idi)

#Result 112, this is correct! The data is load correct

112


In [41]:
dup=mergedDF.groupby(mergedDF.columns).count().where('count > 1')
dup.show()
#there is not dupplicates

In [49]:
rows=mergedDF.count()
print('number of rows:',rows)
#167253289

167253289


In [50]:
dup2=mergedDF.groupby(['time_id','seconds_in_bucket','stock_id']).count().where('count > 1')
dup2.show()
#there is not dupplicates

+-------+-----------------+--------+-----+
|time_id|seconds_in_bucket|stock_id|count|
+-------+-----------------+--------+-----+
+-------+-----------------+--------+-----+



Counting null values

In [53]:
from pyspark.sql.functions import col,isnan, when, count

mergedDF.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in mergedDF.columns]
   ).show()
#There is not null values

+-------+-----------------+----------+----------+----------+----------+---------+---------+---------+---------+--------+
|time_id|seconds_in_bucket|bid_price1|ask_price1|bid_price2|ask_price2|bid_size1|ask_size1|bid_size2|ask_size2|stock_id|
+-------+-----------------+----------+----------+----------+----------+---------+---------+---------+---------+--------+
|      0|                0|         0|         0|         0|         0|        0|        0|        0|        0|       0|
+-------+-----------------+----------+----------+----------+----------+---------+---------+---------+---------+--------+

